# Recipe → Colab Parameters

**What this notebook does**
- Clone `GGUF-Tool-Suite`, accept a `.recipe` upload, run `recipe_to_colab_params.py` and **display** the resulting Colab pipeline parameters in the notebook output.

**Quick steps**
1. Run cells in order: **Clone**, **Upload**, **Run**.
2. Upload a `.recipe` file when prompted.
3. View the generated pipeline parameters in the output of the **Run** cell.

**Note:** The produced pipeline parameters are intended to be used with [quant_recipe_pipeline.ipynb on Colab](
https://colab.research.google.com/github/Thireus/GGUF-Tool-Suite/blob/main/quant_recipe_pipeline.ipynb).

**Aim & caveats:** This tool attempts to convert a `.recipe` into a Colab parameter list that can reproduce (or improve) recipes using newer versions of `quant_assign.py`. It is experimental — parsing errors can occur, so please check the output before using it.

In [ ]:
%%bash
set -e
REPO="https://github.com/Thireus/GGUF-Tool-Suite.git"
DIR="GGUF-Tool-Suite"
echo "Removing any existing $DIR (if present)..."
rm -rf "$DIR"
echo "Cloning $REPO ..."
git clone "$REPO" "$DIR"
echo "Repository cloned into $DIR"


In [ ]:
from google.colab import files
print("Upload a .recipe file (file must end with .recipe)")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded.")
fname = next(iter(uploaded))
if not fname.lower().endswith(".recipe"):
    raise SystemExit("Uploaded file does not have a .recipe extension: " + fname)
print("Uploaded:", fname)


In [ ]:
# Run recipe_to_colab_params.py from the cloned repository on the uploaded .recipe file.
# Display the generated Python parameters snippet in the notebook output.
import os, subprocess, shlex, sys
from IPython.display import display, Markdown

try:
    fname  # must be set by upload cell
except NameError:
    raise SystemExit("fname not found. Did you run the upload cell?")

repo_dir = "GGUF-Tool-Suite"
script_path = os.path.join(repo_dir, "recipe_to_colab_params.py")
if not os.path.exists(script_path):
    raise SystemExit(f"Cannot find recipe_to_colab_params.py at {script_path}")

outfile = fname + ".colab_params.py"
cmd = f"python3 {shlex.quote(script_path)} {shlex.quote(fname)} -o {shlex.quote(outfile)}"
print("Running:", cmd)
proc = subprocess.run(cmd, shell=True, capture_output=True, text=True)
print(proc.stdout)
if proc.returncode != 0:
    print(proc.stderr, file=sys.stderr)
    raise SystemExit(f"Script failed with exit code {proc.returncode}")

print("\nGenerated parameters file:", outfile)
# Read and display the content neatly
with open(outfile, "r", encoding="utf-8") as f:
    content = f.read()

# Display as a Python code block for easy copying
display(Markdown("**Colab pipeline parameters produced:**"))
display(Markdown(f"```python\n{content}\n```"))
